In [4]:
from google_auth_oauthlib.flow import InstalledAppFlow

flow = InstalledAppFlow.from_client_secrets_file(
    'client_secret.json',
    scopes=[
        'https://www.googleapis.com/auth/drive.file',
        'https://www.googleapis.com/auth/spreadsheets'
    ])
credentials = flow.run_local_server(host='localhost',
    port=8080, 
    authorization_prompt_message='Please visit this URL: {url}', 
    success_message='The auth flow is complete; you may close this window.',
    open_browser=True)

Please visit this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=251072080257-ki89qocbc9boe3cf054b5g8qig8sintm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=Q4kAuOQQW2DtvqC77g3K2XlgRTuKrA&access_type=offline


In [14]:
import urllib.parse
import requests
import string

def col2num(col):
    num = 0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A')) + 1
    return num


def download_pdf(google_sheet_id, sheet_gid, range_rows, range_columns, file_name):
    access_token = credentials.token
    q = {
        'format': 'pdf',
        'size': 'letter',
        'portrait': 'true',
        'fitw': 'true',
        'sheetnames': 'true',
        'printtitle': 'true',
        'pagenumbers': 'CENTER',
        'gridlines': 'false',
        'printnotes': 'false',
        'fzr': 'false',
        'gid': sheet_gid,
        'r1': range_rows[0] - 1,  # Top of range, 0-indexed
        'r2': range_rows[1],  # Bottom of range, 1-indexed
        'c1': col2num(range_columns[0]) - 1,  # Left of range, 0-indexed
        'c2': col2num(range_columns[1])  # Right of range, 1-indexed
    }
    query_parameters = urllib.parse.urlencode(q)
    url = f'https://docs.google.com/spreadsheets/d/{google_sheet_id}/export?{query_parameters}'
    headers = {'Authorization': 'Bearer ' + access_token}
    res = requests.get(url, headers=headers)
    with open(file_name, "wb") as f:
        f.write(res.content)
    print(f"File saved as: \"{file_name}\"")


def get_cell_values(service, google_sheet_id, range_string):
    result = (
        service.spreadsheets()
        .values()
        .get(spreadsheetId=google_sheet_id, range=range_string)
        .execute()
    )
    rows = result.get("values", [])
    if rows:
        return rows
    else:
        return None


def update_cell(service, google_sheet_id, cell, value):
    body = {
        "values": [
            [
                value
            ],
        ]
    }
    result = (
        service.spreadsheets()
        .values()
        .update(
            spreadsheetId=google_sheet_id,
            range=cell,
            valueInputOption='USER_ENTERED',
            body=body,
        )
        .execute()
    )
    print(f"Cell {cell} updated to: \"{value}\"")

In [15]:
import time
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

google_sheet_id = "1Dc7P46n_aElQeCA21TAncLKSk6BVfQEsVxYDzcNbtv0"

unit_name_cell = "'Pio Pico'!AA3"
names_range = "'Pio Pico'!Y8:Y200"

sheet_gid = '922725730'  # the number from gid= part of the URL of the sheet
units = [
    '',
    'Crew 0461',
    'Crew 0546',
    'Pack 0018 (B)',
    'Pack 0045 (F)',
    'Pack 0072 (F)',
    'Pack 0219 (F)',
    'Pack 0242 (F)',
    'Pack 0438 (F)',
    'Pack 0476 (F)',
    'Pack 0529 (F)',
    'Pack 0553 (B)',
    'Pack 0558 (F)',
    'Pack 0688 (B)',
    'Pack 0805 (B)',
    'Pack 0919 (F)',
    'Pack 0924 (F)',
    'Pack 4914 (F)',
    'Troop 0002 (B)',
    'Troop 0033 (B)',
    'Troop 0033 (G)',
    'Troop 0054 (B)',
    'Troop 0072 (B)',
    'Troop 0219 (B)',
    'Troop 0330 (B)',
    'Troop 0375 (B)',
    'Troop 0394 (B)',
    'Troop 0394 (G)',
    'Troop 0438 (B)',
    'Troop 0441 (B)',
    'Troop 0468 (B)',
    'Troop 0476 (B)',
    'Troop 0546 (B)',
    'Troop 0688 (B)',
    'Troop 0693 (B)',
    'Troop 0693 (G)',
    'Troop 0841 (B)',
    'Troop 0888 (B)',
    'Troop 0919 (B)',
    'Troop 0924 (B)',
    'Troop 0924 (G)',
    'Troop 0985 (B)',
    'Troop 1005 (B)',
    'Troop 1726 (B)',
    'Troop 2019 (G)',
    'Troop 4914 (B)',
    'Troop 5530 (B)'
]

try:
    with build('sheets', 'v4', credentials=credentials) as service:
        for unit in units:
            update_cell(service, google_sheet_id, unit_name_cell, unit)
            unit_name = get_cell_values(service, google_sheet_id, unit_name_cell)
            if unit_name:
                unit_name = unit_name[0][0]
            else:
                unit_name = "MBC and District"

            names = get_cell_values(service, google_sheet_id, names_range)
            export_range_rows = (2, len(names) + 7)
            export_range_columns = ('Y', 'AD')
            file_name = f"GLAAC AB506 Report - Pio Pico - {unit_name}.pdf"
            download_pdf(google_sheet_id, sheet_gid, export_range_rows, export_range_columns, file_name)
            time.sleep(0.01)

except HttpError as error:
    print(f"An error occurred: {error}")

Cell 'Pio Pico'!AA3 updated to: ""
File saved as: "GLAAC AB506 Report - Pio Pico - MBC and District.pdf"
